<a href="https://colab.research.google.com/github/evanwilliamsconsulting/Kaggle_Feedback_Prize/blob/main/Kaggle_Student_Essays_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
import os

In [ ]:
from os import listdir

In [ ]:
# https://stackoverflow.com/questions/48376580/google-colab-how-to-read-data-from-my-google-drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
DATA = "/content/drive/MyDrive"

In [ ]:
corpus_directory = os.path.join(DATA,'Kaggle','data','train')

In [ ]:
root_directory = os.path.join(DATA,'Kaggle')

In [ ]:
datasets_directory = os.path.join(DATA,'Kaggle','datasets')

In [ ]:
train_filename = "train.csv"

In [ ]:
train_filepath = os.path.join(root_directory,train_filename)

In [ ]:
essay_files = listdir(corpus_directory)

In [ ]:
len(essay_files)

In [ ]:
essay_filename = essay_files[664]

In [ ]:
essay_filepath = os.path.join(corpus_directory,essay_filename)

In [ ]:
essay_text = pd.read_table(essay_filepath,header=0,engine='python',error_bad_lines=False)

In [ ]:
# to bring in the tags

In [ ]:
train_df = pd.read_csv(train_filepath,header=0,engine='python')

In [ ]:
train_df.head()

In [ ]:
print(len(train_df))

In [ ]:
def retrieveWordage(essay_id):
  essay_filename = essay_id + ".txt"
  essay_filepath = os.path.join(corpus_directory,essay_filename)
  essay_text = pd.read_table(essay_filepath,header=0,engine='python',error_bad_lines=False)
  essay_text.columns = ['narrative']
  return essay_text

In [ ]:
wordage_test = retrieveWordage("0027FC00C35B")

In [ ]:
def cleanText2(item):
  item2 = ""
  for ch in item:
      if ch == ' ' or (ch >= 'a' and ch <= 'z') or (ch >= 'A' and ch <= 'Z') or (ch >= '0' and ch <= '9'):
          if ch >= 'A' and ch <= 'Z':
              item2 += ch.lower()
          else:
              item2 += ch
  return item2

In [ ]:
def retrieveEssay(essay_id):
  essay_text = retrieveWordage(essay_id)
  essay_count = len(essay_text)
  essay = ""
  for i in range(0,essay_count-1):
      essay += cleanText2(essay_text.iloc[i,0])
  return essay

In [ ]:
def retrieveRawEssay(essay_id):
  essay_text = retrieveWordage(essay_id)
  essay_count = len(essay_text)
  essay = ""
  for i in range(0,essay_count-1):
      item = essay_text.iloc[i,0]
      essay += item
  return essay

In [ ]:
essay_test = retrieveEssay("423A1CA112E2")

In [ ]:
from tensorflow.data import Dataset

In [ ]:
from tensorflow.data import TextLineDataset

In [ ]:
def retrieveDataset(essay_id):
  essay_array = []
  essay_text = retrieveWordage(essay_id)
  essay_count = len(essay_text)
  #essay = ""
  for i in range(0,essay_count-1):
      essay_array.append(cleanText2(essay_text.iloc[i,0]))
  #essay = essay_text.iloc[0,0]
  dataset = Dataset.from_tensor_slices(essay_array)
  return dataset

In [ ]:
text_dataset = retrieveDataset("423A1CA112E2")

In [ ]:
print(text_dataset)

In [ ]:
for element in text_dataset:
  print(element)

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [ ]:
max_features = 5000  # Maximum vocab size.
max_len = 500  # Sequence length to pad the outputs to.b

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)

In [ ]:
#text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_dataset)

In [ ]:
# Create the layer.
vectorize_layer = TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

In [ ]:
# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for large
# datasets this means we're not keeping spare copies of the dataset.
vectorize_layer.adapt(text_dataset)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow import string as tf_string

In [ ]:
# Create the model that uses the vectorize text layer
model = Sequential()

# Start by creating an explicit input layer. It needs to have a shape of
# (1,) (because we need to guarantee that there is exactly one string
# input per batch), and the dtype needs to be 'string'.
model.add(Input(shape=(1,), dtype=tf_string))

# The first layer in our model is the vectorization layer. After this
# layer, we have a tensor of shape (batch_size, max_len) containing vocab
# indices.
model.add(vectorize_layer)

In [ ]:
# Now, the model can map strings to integers, and you can add an embedding
# layer to map these integers to learned embeddings.
input_data = [["modern humans today"], ["phone more than"]]
model.predict(input_data)

In [ ]:
test = retrieveRawEssay("423A1CA112E2")
print(test)

In [ ]:
def retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end):
  essay_text = retrieveRawEssay(essay_id)
  essay_count = len(essay_text)
  i = 0
  discourse_text = ""
  for ch in essay_text:
     #print(ch)
     if i >= discourse_start -1 and i <= discourse_end:
        discourse_text += ch
     i += 1
  return cleanText2(discourse_text)

In [ ]:
discourse = retrieveDiscourse("423A1CA112E2",0,8,229)

In [ ]:
print(discourse)

In [ ]:
print(len(discourse))

In [ ]:
input_data = [discourse]
model.predict(input_data)

In [ ]:
# https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/pretrained_word_embeddings.ipynb#scrollTo=Q6j-LRMFc-AR

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip


In [ ]:
#!unzip -q glove.6B.zip

In [ ]:
path_to_glove_file = os.path.join(datasets_directory,"glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
print(voc)

In [ ]:
print(word_index)

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant

In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import Model

In [ ]:
# The class names should be the categories we are looking for.
# right now use 1 and 0!
class_names = ["1","0"]

In [ ]:
int_sequences_input = Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = Model(int_sequences_input, preds)
model.summary()

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_test = vectorizer(np.array([[s] for s in test_samples])).numpy()

In [ ]:
print(len(train_labels))

In [ ]:
y_train = np.array([[s] for s in train_labels])
y_test = np.array([[s] for s in test_labels])

In [ ]:
train_labels = range(0,class_index)
test_labels = range(0,class_index)

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

In [ ]:
#model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))
model.fit(x_train, y_train, batch_size=128, epochs=20)
#model.fit(x_train,y_train)

In [ ]:
y_result = model.predict(x_test)

In [ ]:
print(y_result)

In [ ]:
y0 = y_result[0]

In [ ]:
print(np.round(y0,1))

In [ ]:
yt0 = y_test[0]

In [ ]:
print(yt0)

In [ ]:
tokens_essay = tokenize_text(test)

In [ ]:
print(tokens_essay)

In [ ]:
tokens_discourse = tokenize_text(discourse)

In [ ]:
print(tokens_discourse)

In [ ]:
discourses = train_df[train_df["id"] == "423A1CA112E2"]

In [ ]:
print(discourses)

In [ ]:
discourses.shape

In [ ]:
discourse1 = discourses.iloc[0]

In [ ]:
print(discourses)

In [ ]:
print(discourse1)

In [ ]:
prediction_string = discourse1["predictionstring"]

In [ ]:
print(prediction_string)

In [ ]:
discourse_text = discourse1["discourse_text"]

In [ ]:
print(discourse_text)

In [ ]:
###### Example from TESU DSI 611 Final Project

In [ ]:
##### Select Discourse Type = Lead

In [ ]:
train_df.info()

In [ ]:
essays_group = train_df.groupby(['id'])

In [ ]:
essay_test = retrieveEssay("0027FC00C35B")

In [ ]:
class_dict = {}
samples = []
labels = []
class_index = 0
for essay_id, essay_pd in essays_group:  
  essay = retrieveEssay(essay_id)
  discourse_count = len(essay_pd)
  for i in range(0,discourse_count):
    item = essay_pd.iloc[i,]
    discourse_id = item['discourse_id']
    discourse_start = item['discourse_start']
    discourse_end = item['discourse_end']
    discourse_type = item['discourse_type']
    discourse_type = discourse_type.lower()
    if class_dict.get(discourse_type) == None:
      class_name_index = class_index
      class_dict[discourse_type] = class_name_index
      class_index += 1
    else:
      class_name_index = class_dict.get(discourse_type)
    discourse = retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end)
    samples.append(discourse)
    labels.append(class_name_index)

In [ ]:
# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
test_samples = samples[-num_validation_samples:]


In [ ]:
train_labels = labels[0:18300]
test_labels = labels[18300:22875]

In [ ]:
print(num_validation_samples)

In [ ]:
print(len(samples))

In [ ]:
print(len(samples)-num_validation_samples)

In [ ]:
print(len(train_labels))

In [ ]:
print(len(test_labels))

In [ ]:
print(len(labels))

In [ ]:
print(samples)

In [ ]:
print(labels)

In [ ]:
print(len(samples))

In [ ]:
print(len(labels))

In [ ]:
print(class_dict)

In [ ]:
class_names = []
for key in class_dict:
  print(key)
  class_names.append(key)

In [ ]:
print(class_names)

In [ ]:
#lead_df = train_df[train_df['discourse_type']=='Lead']

In [ ]:
print(lead_df)

In [ ]:
lead_df.shape

In [ ]:
sample = lead_df.iloc[0,:]

In [ ]:
print(sample)

In [ ]:
lead_length = len(lead_df)

In [ ]:
print(lead_length)

In [ ]:
# This is the example
# samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

In [ ]:
essays = []
leads = []
train_samples = []
lead_length = 200
class_names = []
class_index = 0
for i in range(0,lead_length-1):
  essay_id = lead_df.iloc[i,0]
  discourse_id = lead_df.iloc[i,1]
  discourse_start = lead_df.iloc[i,2]
  discourse_end = lead_df.iloc[i,3]
  try:
      essay = retrieveEssay(essay_id)
      discourse = retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end)
      essays.append(essay)
      train_samples.append(essay)
      leads.append(discourse)
      class_names.append(essay_id)
      class_index += 1
  except:
      print(essay_id)

In [ ]:
essays_test = []
leads_test = []
test_samples = []
test_names = []
test_index = 0
for i in range(220,240):
  essay_id = lead_df.iloc[i,0]
  discourse_id = lead_df.iloc[i,1]
  discourse_start = lead_df.iloc[i,2]
  discourse_end = lead_df.iloc[i,3]
  try:
      essay = retrieveEssay(essay_id)
      discourse = retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end)
      essays_test.append(essay)
      test_samples.append(essay)
      leads_test.append(discourse)
      test_names.append(essay_id)
      test_index += 1
  except:
      print(essay_id)

In [ ]:
print(essays)

In [ ]:
print(leads)

In [ ]:
print(len(essays))
print(len(leads))

In [ ]:
print(len(essays_test))
print(len(leads_test))

In [ ]:
maxlen = 0
for essay in essays:
  essay_length = len(essay)
  #print(essay_length)
  if essay_length > 3000:
    print(essay_length)
  if maxlen < essay_length:
    maxlen = essay_length
print(maxlen)

In [ ]:
def returnMaxLength(essay_array):
  maxlen = 0
  for essay in essay_array:
    essay_length = len(essay)
    if maxlen < essay_length:
      maxlen = essay_length
  return maxlen

In [ ]:
print(returnMaxLength(essays))
print(returnMaxLength(leads))
print(returnMaxLength(essays_test))
print(returnMaxLength(leads_test))

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [ ]:
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(essays)
t.word_index['<PAD>'] = 0

In [ ]:
# transform train set using the tokenizer
train_sequences = t.texts_to_sequences(essays)

In [ ]:
# transform test set using the tokenizer
test_sequences = t.texts_to_sequences(essays_test)

In [ ]:
train_sequence_leads = t.texts_to_sequences(leads)
test_sequence_leads = t.texts_to_sequences(leads_test)

In [ ]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

In [ ]:
maxlen = 0
for seq in train_sequences_padded:
  seqlen = len(seq)
  print(seqlen)
  if maxlen < seqlen:
    maxlen = seqlen
print(maxlen)

In [ ]:
def returnSequenceLength(sequence):
  maxlen = 0
  for seq in sequence:
    seqlen = len(seq)
    if maxlen < seqlen:
      maxlen = seqlen
  return maxlen

In [ ]:
print(returnSequenceLength(train_sequences))
print(returnSequenceLength(test_sequences))

In [ ]:
train_pad_length = returnSequenceLength(train_sequences)
test_pad_length = returnSequenceLength(test_sequences)

In [ ]:
train_pad_length_leads = returnSequenceLength(train_sequence_leads)
train_pad_length_leads_test = returnSequenceLength(test_sequence_leads)

In [ ]:
print(train_pad_length_leads)
print(train_pad_length_leads_test)

In [ ]:
i = 0
train_sequence_example = train_sequences[0]
train_sequence_leads_example = train_sequence_leads[0]
print(train_sequence_example)
print(train_sequence_leads_example)

In [ ]:
i = 0
j = 0
k = 0
match_flag = 0
output_array = []
len_leads_example = len(train_sequence_leads_example)
len_essay_example = len(train_sequence_example)
for item in train_sequence_example:
    i = k
    while i < len_leads_example - 1 and match_flag == 0:
        item2 = train_sequence_leads_example[i]
        if item == item2:
            output_array.append(1)
            k = i
            match_flag = 1
        i += 1
    if match_flag == 0:
        output_array.append(0)
    else:
        match_flag = 0 # reset the match flag

In [ ]:
print(output_array)

In [ ]:
print(len(train_sequence_example))
print(len(train_sequence_leads_example))
print(len(output_array))

In [ ]:
def returnOutputArray(sequence_essay,sequence_lead):
  i = 0
  j = 0
  k = 0
  match_flag = 0
  output_array = []
  len_leads_example = len(sequence_lead)
  len_essay_example = len(sequence_essay)
  for item in sequence_essay:
      i = k
      while i < len_leads_example - 1 and match_flag == 0:
          item2 = sequence_lead[i]
          if item == item2:
              output_array.append(1)
              k = i
              match_flag = 1
          i += 1
      if match_flag == 0:
          output_array.append(0)
      else:
          match_flag = 0 # reset the match flag
  return(output_array)

In [ ]:
output_array_example = returnOutputArray(train_sequence_example,train_sequence_leads_example)

In [ ]:
print(output_array_example)

In [ ]:
number_sequences = len(train_sequences)
number_sequences_leads = len(train_sequence_leads)
print(number_sequences)
print(number_sequences_leads)

In [ ]:
y_train = []
for i in range(0,181):
  train_seq = train_sequences[i]
  train_leads_seq = train_sequence_leads[i]
  output_array = returnOutputArray(train_seq,train_leads_seq)
  y_train.append(output_array)

In [ ]:
print(y_train[1])

In [ ]:
number_sequences = len(test_sequences)
number_sequences_leads = len(test_sequence_leads)
print(number_sequences)
print(number_sequences_leads)

In [ ]:
y_test = []
for i in range(0,16):
  test_seq = test_sequences[i]
  test_leads_seq = test_sequence_leads[i]
  output_array = returnOutputArray(test_seq,test_leads_seq)
  y_test.append(output_array)

In [ ]:
print(y_test)

In [ ]:
def returnPaddedSequence(sequence,pad_length):
  seqlen = len(sequence)
  return_sequence = []
  i = 0
  for item in sequence:
    return_sequence.append(item)
    i += 1  
  if i < pad_length:
    while i < pad_length:
      return_sequence.append(0)
      i += 1
  return return_sequence

In [ ]:
def returnPaddedSequences(sequences,pad_length):
  return_sequences = []
  for seq in sequences:
    return_sequences.append(returnPaddedSequence(seq,pad_length))
  return return_sequences

In [ ]:
print(len(train_sequences))
train_sequences_padded = returnPaddedSequences(train_sequences,train_pad_length)
print(len(train_sequences_padded))

In [ ]:
print(len(test_sequences))
test_sequences_padded = returnPaddedSequences(test_sequences,test_pad_length)
print(len(test_sequences_padded))

In [ ]:
print(returnSequenceLength(train_sequences))
print(returnSequenceLength(train_sequences_padded))

In [ ]:
print(returnSequenceLength(test_sequences))
print(returnSequenceLength(test_sequences_padded))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens = [len(s) for s in train_sequences_padded]
test_lens = [len(s) for s in test_sequences_padded]

fig, ax = plt.subplots(1,2, figsize=(12, 6))
h1 = ax[0].hist(train_lens)
h2 = ax[1].hist(test_lens)

In [ ]:
# while 250 is long should be a safe bet
MAX_SEQUENCE_LENGTH = 1400

In [ ]:
# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences_padded, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences_padded, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

In [ ]:
le = LabelEncoder()
# positive -> 1, negative -> 0
num_classes=2 

In [ ]:
def return_y_vector(input_sequence,lead_sequence)

In [ ]:
y_train2 = np.array(y_train)
y_test2 = np.array(y_test)

In [ ]:
nparrs = []
for arr in y_train2:
  nparr = np.array(arr).astype("float32")
  nparrs.append(nparr)
y_train3 = np.array(nparrs)

In [ ]:
VOCAB_SIZE = len(t.word_index)

In [ ]:
EMBED_SIZE = 300
EPOCHS = 10
BATCH_SIZE = 128

model = Sequential()
model.add(Embedding(VOCAB_SIZE,EMBED_SIZE,input_length=MAX_SEQUENCE_LENGTH))

model.add(Conv1D(filters=128,kernel_size=4,activation='relu',padding='same'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64,kernel_size=4,activation='relu',padding='same'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32,kernel_size=4,activation='relu',padding='same'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True,verbose=1)

In [ ]:
# X_train,y_train

In [ ]:
X_train = np.array(train_sequences_padded)

In [ ]:
nparrs = []
for item in train_sequences_padded:
  nparr = np.array(item).astype("float32")
  nparrs.append(nparr)

In [ ]:
result_X = np.array(nparrs).astype("float32")

In [ ]:
result_X.shape

In [ ]:
model.fit(result_X,y_train3,validation_split=0.1,epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=[es],verbose=1)

In [ ]:
# https://www.tensorflow.org/guide/keras/rnn#built-in_rnn_layers_a_simple_example
# Looks like I have to normalize the essay length and discourse length!
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()